In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import pymc3 as pm
import arviz as az
from scipy import stats
pd.set_option('display.max_columns', 500)
import warnings
from scipy.stats import spearmanr
import sys
warnings.filterwarnings("ignore")

import logging
logger = logging.getLogger('pymc3')
logger.setLevel(logging.CRITICAL)
%matplotlib inline

In [3]:
sys.path.append("/Users/ailin/Documents/grid-dynamics/DS/data_cleaning/")

In [4]:
from data_preparation_faster import prepare_triplogs, prepare_tripdatas, get_clean_triplogs, get_clean_tripdatas, get_and_clean_tripdatas_triplogs_merge

# Data preparation

In [5]:
%%time
from collections import defaultdict
filtration_dict=defaultdict(lambda: True)
# no need to drop tsp mode
filtration_dict["tspmode_not_normal_test"] = False

triplogs = pd.read_csv('../../../trips/2020_11_8_to_2020_11_14/CVP/triplogs.csv')
tripdatas = pd.read_csv('../../../trips/2020_11_8_to_2020_11_14/CVP/tripdatas.csv')

triplogs = prepare_triplogs(triplogs)
tripdatas = prepare_tripdatas(tripdatas)

# takes about 10 minutes to run

tripdatas = get_clean_tripdatas(tripdatas) # cleaning tripdatas first is important
tripdatas_df_good = tripdatas[tripdatas["is_good_for_study"]] # remove messy tripdatas now so it doesnt affect triplogs cleaning
triplogs = get_clean_triplogs(triplogs, tripdatas_df_good, filtration_dict)
triplogs_clean = triplogs[triplogs["is_good_for_study"]]

# triplogs_clean = add_columns_to_prepared_triplogs(triplogs_clean, tripdatas_df_good)

drop invalid
drop starttime >= endtime
drop endstatus not completed
drop negative duration
drop <=70% stops hit
drop <=70% breadcrumbs
drop duplicated rows
drop tspon no tsp requests
drop 3 stds anomalies
CPU times: user 2min 56s, sys: 2min 21s, total: 5min 17s
Wall time: 6min 50s


In [6]:
# drop all events except arrive/depart/request
tripdatas_df_good = tripdatas_df_good[tripdatas_df_good.event.isin(['stop depart', 'TSP request','stop arrive', 'routename'])]

In [7]:
# merge tripdatas with triplogs
# logdatas = tripdatas_df_good.merge(triplogs[['uid', 'direction']], on='uid', how='inner')
logdatas = get_and_clean_tripdatas_triplogs_merge(tripdatas_df_good, triplogs_clean[['uid', 'direction', 'routename']])
logdatas.rename({'routename_x': 'routename'}, axis=1, inplace=True)

In [15]:
logdatas.head()

,__v,_id,deviceid,dir,distance,dwelltime,emitterid,event,headway,loc,logid,mph,routename,stopindex,stopname,time,tracktime,traveltime,unloc,date_ran,uid,is_good_for_study,direction,routename_y,routename_direction
0,0,{oid=5fa7891e815102901719b9b0},4010KO2052,177.2,NaN,NaN,NaN,TSP request,0.0,"{coordinates=[-122.476716666667, 37.7567366666...",20201107-t5,14.154594,28,NaN,NaN,2020-11-08 01:17:58.364,0.0,NaN,"{coordinates=[], type=Point}",2020-11-08,4010KO2052_20201107-t5,True,outbound,28,28_outbound
1,0,{oid=5fa7891e815102901719b91d},4010KO2052,176.6,11.0,0.0,NaN,stop arrive,NaN,"{coordinates=[-122.476136666667, 37.74829], ty...",20201107-t5,20.483884,28,13.0,19th Ave & Quintara St:,2020-11-08 01:20:39.701,-6.0,39.0,"{coordinates=[], type=Point}",2020-11-08,4010KO2052_20201107-t5,True,outbound,28,28_outbound
2,0,{oid=5fa7891e815102901719b96a},4010KO2052,167.9,27.0,21.0,NaN,stop depart,NaN,"{coordinates=[-122.476523333333, 37.7539366666...",20201107-t5,7.825304,28,11.0,19th Ave & Noriega St:,2020-11-08 01:19:14.292,-8.0,0.0,"{coordinates=[], type=Point}",2020-11-08,4010KO2052_20201107-t5,True,outbound,28,28_outbound
3,0,{oid=5fa7891e815102901719b743},4010KO2052,178.5,234.0,1.0,NaN,stop depart,NaN,"{coordinates=[-122.475416666667, 37.72092], ty...",20201107-t5,24.281458,28,20.0,19th Ave & Holloway Ave,2020-11-08 01:29:47.754,-3.0,0.0,"{coordinates=[], type=Point}",2020-11-08,4010KO2052_20201107-t5,True,outbound,28,28_outbound
4,0,{oid=5fa7891e815102901719b693},4010KO2052,179.2,255.0,0.0,NaN,stop depart,NaN,"{coordinates=[-122.471293333333, 37.7106216666...",20201107-t5,34.983712,28,23.0,Junipero Serra Blvd / S.F. Golf Club,2020-11-08 01:33:09.144,-5.0,0.0,"{coordinates=[], type=Point}",2020-11-08,4010KO2052_20201107-t5,True,outbound,28,28_outbound


In [16]:
# we need to group by routename and direction, let's do it now
logdatas['routename_direction'] = logdatas['routename'] +'_'+ logdatas['direction']

In [17]:
from collections import defaultdict

def get_uid_to_curbreadcrumbs(tripdatas_df_good):
    """ make a {"routename_direction": df_tripdatas} """
    uid_to_curbreadcrumbs = {}
    tripdatas_df_good = tripdatas_df_good[tripdatas_df_good["event"].isin(['stop depart', 'TSP request','stop arrive'])].sort_values("time")
    for routename_direction in tripdatas_df_good.routename_direction.unique():
        # df_tripdatas for speciphic routename_direction
        t_df = tripdatas_df_good[tripdatas_df_good.routename_direction == routename_direction]
        if sum(t_df.event == 'TSP request') > 0:
            # start df with departing from the first bus stop
            while t_df.event.iloc[0] != 'stop depart':
                t_df = t_df.iloc[1:]
            if len(t_df.index) >= 2:
                uid_to_curbreadcrumbs[routename_direction] = t_df

    return uid_to_curbreadcrumbs

In [18]:
uid_to_curbreadcrumbs = get_uid_to_curbreadcrumbs(logdatas)

In [19]:
# check that it worked
uid_to_curbreadcrumbs['9_outbound'].head(2)

,__v,_id,deviceid,dir,distance,dwelltime,emitterid,event,headway,loc,logid,mph,routename,stopindex,stopname,time,tracktime,traveltime,unloc,date_ran,uid,is_good_for_study,direction,routename_y,routename_direction
379392,0,{oid=5faa3d9b80ad8ba417b63117},4010KK2096,224.06,281.0,0.0,NaN,stop depart,NaN,"{coordinates=[-122.396468333333, 37.7935733333...",20201107-t19,14.534351,9,2.0,Market St & Drumm St,2020-11-08 00:24:56.453,0.0,0.0,"{coordinates=[], type=Point}",2020-11-08,4010KK2096_20201107-t19,True,outbound,9,9_outbound
379428,0,{oid=5faa3d9b80ad8ba417b63063},4010KK2096,241.66,63.0,0.0,NaN,stop arrive,NaN,"{coordinates=[-122.401366666667, 37.7893216666...",20201107-t19,13.935946,9,4.0,Market St & 2nd St,2020-11-08 00:28:39.386,1.0,223.0,"{coordinates=[], type=Point}",2020-11-08,4010KK2096_20201107-t19,True,outbound,9,9_outbound


In [20]:
# collect a table with segments durations
route_direction_start_depart_duration_tsp = [] # future df
for route_direction in uid_to_curbreadcrumbs:
    tsp = False
    depart_time = 0
    for idx, row in uid_to_curbreadcrumbs[route_direction].iterrows():
        event = row['event']
        if event == 'stop depart':
            depart_time = row['time']
            # starting bus stop
            start = row['stopname']
        elif event == 'stop arrive':
            route_direction_start_depart_duration_tsp.append( 
                [row.routename, row.direction, start, row.stopname, (row.time - depart_time).total_seconds(), tsp, depart_time, row.time]
            )
            tsp = False
        elif event == 'TSP request':
            tsp = True
                

In [21]:
route_direction_start_depart_duration_tsp[:2]

[['1',
  'outbound',
  'Sacramento St & Davis St',
  'Sacramento St & Battery St',
  92.992,
  False,
  Timestamp('2020-11-08 00:01:03.823000'),
  Timestamp('2020-11-08 00:02:36.815000')],
 ['1',
  'outbound',
  'Sacramento St & Battery St',
  'Sacramento St & Sansome St',
  61.722,
  False,
  Timestamp('2020-11-08 00:02:37.838000'),
  Timestamp('2020-11-08 00:03:39.560000')]]

In [22]:
# form a df
segments_df = pd.DataFrame.from_records(route_direction_start_depart_duration_tsp)
segments_df.columns = ['routename', 'direction', 'start', 'end', 'duration', 'tspmode', 'starttime', 'endtime']
segments_df['starthour'] = segments_df.starttime.dt.hour

In [23]:
segments_df.head()

,routename,direction,start,end,duration,tspmode,starttime,endtime,starthour
0,1,outbound,Sacramento St & Davis St,Sacramento St & Battery St,92.992,False,2020-11-08 00:01:03.823,2020-11-08 00:02:36.815,0
1,1,outbound,Sacramento St & Battery St,Sacramento St & Sansome St,61.722,False,2020-11-08 00:02:37.838,2020-11-08 00:03:39.560,0
2,1,outbound,Sacramento St & Sansome St,Sacramento St & Montgomery St,17.147,False,2020-11-08 00:03:40.583,2020-11-08 00:03:57.730,0
3,1,outbound,Sacramento St & Montgomery St,Sacramento St & Kearny St,61.788,False,2020-11-08 00:03:58.753,2020-11-08 00:05:00.541,0
4,1,outbound,Sacramento St & Kearny St,Sacramento St & Grant Ave,49.085,False,2020-11-08 00:05:01.570,2020-11-08 00:05:50.655,0


In [24]:
sum(segments_df.duration < 0) # no troubles???

0

# Hypotheses testing

In [25]:
# mann-whitney u-test
def mannwhitney(triplogs_on, triplogs_off):
    rejected = []
    total = 0
    for idx in triplogs_on.index:
        if idx in triplogs_off.index:
            total += 1
            pval = stats.mannwhitneyu(triplogs_on[idx], triplogs_off[idx], alternative='less').pvalue
            if pval < 0.05:
#                 print(idx, f'route is significally faster with pval {pval:.2f}')
                rejected.append(
                    (pval, idx)
                )
    print(len(rejected), 'of', total, 'rejected', len(rejected)/total)
    return rejected

In [26]:
# student two samples t-test
def student(triplogs_on, triplogs_off):
    rejected = []
    total = 0
    for idx in triplogs_on.index:
        if idx in triplogs_off.index:   
            total += 1
            pval = stats.ttest_ind(triplogs_on[idx], triplogs_off[idx], alternative='less').pvalue
            if pval < 0.05:
                # check same variance assumption (timings of two type of routes must have variance close to each other)
                s1 = np.sqrt(np.var(triplogs_on[idx], ddof=1))
                s2 = np.sqrt(np.var(triplogs_off[idx], ddof=1))
                valid = 'test valid'
                if np.abs(s1 - s2) > 50:
                    valid = 'test invalid'
#                 print(f'{idx} route is significally faster, {valid}, s1 {s1:.2f}, s2 {s2:.2f}')
                rejected.append(
                    (pval, idx)
                )
    print(len(rejected), 'of', total, 'rejected', len(rejected)/len(triplogs_on.index))
    return rejected

In [27]:
# next two funcs are not needed to reimplement
def plot_trips(triplogs_on, triplogs_off, rejected_idxs):
    fig, ax = plt.subplots(len(rejected_idxs), 1,figsize=(15,15), constrained_layout=True)
    for i, (pval, idx)  in enumerate(rejected_idxs):
        title = f'{", ".join(map(str, idx))}, pval {pval:.2f}'
        ax[i].set_title(title)
        ax[i].plot(triplogs_on[idx], label='on')
        ax[i].plot(triplogs_off[idx], label='off')
        ax[i].legend()

In [28]:
def make_df(rejected_idxs, groupby):
    df = [list(x)+[pval] for pval,x in rejected_idxs]
    df = pd.DataFrame(df)
    df.columns = groupby + ['pval']
    df = df.sort_values('pval')
    return df.reset_index(drop=True)

In [29]:
# groupby 
groupby = ['routename', 'direction', 'start', 'end']
# groupby = ['routename', 'direction', 'starthour']

segments_on = segments_df[segments_df.tspmode == True].groupby(groupby)['duration'].apply(list)

segments_off = segments_df[segments_df.tspmode == False].groupby(groupby)['duration'].apply(list)

# drop less that 3 obserationa
segments_off_idx = segments_off.apply(lambda x: len(x) > 3)
segments_off = segments_off[segments_off_idx]
print(segments_off.shape)

segments_on_idx = segments_on.apply(lambda x: len(x) > 3)
segments_on = segments_on[segments_on_idx]
print(segments_on.shape)

(13053,)
(832,)


### mann-whitney 

In [30]:
# mann-whitney
rejected_idxs = mannwhitney(segments_on, segments_off)
# plot_trips(segments_on, segments_off, rejected_idxs)
make_df(rejected_idxs, groupby)

29 of 703 rejected 0.041251778093883355


,routename,direction,start,end,pval
0,38,outbound,Geary Blvd & Laguna St,Geary Blvd & Webster St,0.001608
1,38,outbound,Market St & Montgomery St,Geary St & Kearny St,0.003072
2,8,outbound,4th St & Howard St,4th St & Folsom St,0.003497
3,30,outbound,Chestnut St & Gough St:,Chestnut St & Laguna St,0.003799
4,38,outbound,Geary Blvd & Gough St,Geary Blvd & Laguna St,0.007007
5,JBUS,inbound,Mission St & Appleton Ave,30th St & Dolores St,0.007146
6,43,outbound,South Hill Blvd & Prague St,Munich St & Geneva Ave,0.008281
7,43,inbound,Geneva Ave & Madrid St:,Geneva Ave & Mission St,0.012463
8,38,outbound,Geary Blvd & 28th Ave,Geary Blvd & Collins St,0.012591
9,5,outbound,Mcallister St & Baker St,Fulton St & Masonic Ave,0.013246


### student

In [31]:
# student
rejected_idxs = student(segments_on, segments_off)
# plot_trips(segments_on, segments_off, rejected_idxs)
make_df(rejected_idxs, groupby)

29 of 703 rejected 0.03485576923076923


,routename,direction,start,end,pval
0,38,outbound,Market St & Montgomery St,Geary St & Kearny St,0.001107
1,JBUS,inbound,Mission St & Appleton Ave,30th St & Dolores St,0.005146
2,38,outbound,Geary Blvd & 28th Ave,Geary Blvd & Collins St,0.006497
3,8,outbound,Visitacion Ave & Rutland St,Visitacion Ave & Schwerin St,0.007385
4,38,outbound,Geary Blvd & Laguna St,Geary Blvd & Webster St,0.007929
5,38,outbound,Geary Blvd & Gough St,Geary Blvd & Laguna St,0.010102
6,43,outbound,South Hill Blvd & Prague St,Munich St & Geneva Ave,0.011743
7,5,outbound,Mcallister St & Baker St,Fulton St & Masonic Ave,0.014807
8,30,outbound,Chestnut St & Gough St:,Chestnut St & Laguna St,0.015306
9,9,inbound,Bayshore Blvd & Boutwell St,Bayshore Blvd & Cortland Ave,0.016284


## Grouping with the starthour

In [32]:
# groupby 
groupby = ['routename', 'direction', 'start', 'end', 'starthour']
# groupby = ['routename', 'direction', 'starthour']

segments_on = segments_df[segments_df.tspmode == True].groupby(groupby)['duration'].apply(list)

segments_off = segments_df[segments_df.tspmode == False].groupby(groupby)['duration'].apply(list)

# drop less that 3 obserationa
segments_off_idx = segments_off.apply(lambda x: len(x) > 3)
segments_off = segments_off[segments_off_idx]
print(segments_off.shape)

segments_on_idx = segments_on.apply(lambda x: len(x) > 3)
segments_on = segments_on[segments_on_idx]
print(segments_on.shape)

(14861,)
(286,)


### mann-whitney 

In [33]:
# mann-whitney
rejected_idxs = mannwhitney(segments_on, segments_off)
# plot_trips(segments_on, segments_off, rejected_idxs)
make_df(rejected_idxs, groupby)

4 of 107 rejected 0.037383177570093455


,routename,direction,start,end,starthour,pval
0,9R,inbound,Sunnydale Ave & Bay Shore Blvd:,Bayshore Blvd & Visitacion Ave,1,0.020152
1,38,outbound,Geary Blvd & 12th Ave,Geary Blvd & Park Presidio Blvd,15,0.030301
2,38,outbound,Geary Blvd & 12th Ave,Geary Blvd & Park Presidio Blvd,19,0.047346
3,54,outbound,Geneva Ave & Naples St,Geneva Ave & Madrid St:,2,0.047346


### student 

In [34]:
# student
rejected_idxs = student(segments_on, segments_off)
# plot_trips(segments_on, segments_off, rejected_idxs)
make_df(rejected_idxs, groupby)

3 of 107 rejected 0.01048951048951049


,routename,direction,start,end,starthour,pval
0,9R,inbound,Sunnydale Ave & Bay Shore Blvd:,Bayshore Blvd & Visitacion Ave,1,0.021126
1,9,inbound,San Bruno Ave & Wayland St,San Bruno Ave & Bacon St,15,0.049024
2,38,outbound,Geary Blvd & 20th Ave,Geary Blvd & 22nd Ave,3,0.049978
